# Lookup Table Generation

Let's import the golden ratio package

In [19]:
!pip3 install import_ipynb
import import_ipynb
from golden_ratio import Goldice, INDETERMINATE, COMPLEX_INFINITY
from matplotlib import pyplot
import time
import pandas as pd
import math

Now, let's define some functions:

- compute_closed_set starts from a base of Golder Ratio values and obtains a set applying sum and moltiplication.
- get_commonest takes the most common elements in the closed_set.
- compute_new_base uses the previous defined functions to get a new base.
- bits_needed returns the bits needed to represent an input x.

In [40]:
def compute_closed_set(base : set):
    base_list = list(base)
    final_list = []
    for extern_element in base_list:
        for inner_element in base_list:
            x = extern_element + inner_element
            #print("Sum:", x)
            if ((not x in base) and (x != INDETERMINATE)):
                #print(x, "added")
                final_list.append(x)
            x = extern_element * inner_element
            #print("Product:", x)
            if ((not x in base) and (x != INDETERMINATE)):
                #print(x, "added")
                final_list.append(x)
    return final_list

def get_commonest(closed_set: list):
    commonest_map = dict()
    for elem in closed_set:
        if not elem in commonest_map:
            commonest_map[elem] = 0
        commonest_map[elem] = commonest_map[elem] + 1
    max_value = 0
    for elem in closed_set:
        if commonest_map[elem] > max_value:
            max_value = commonest_map[elem]
    return set([elem for elem in closed_set if commonest_map[elem] == max_value])

def compute_new_base(old_base: set):
    closed_set = compute_closed_set(old_base)
    commonest = get_commonest(closed_set)
    base = old_base.union(commonest)
    return base

def bits_needed(x):
    return len(bin(x).replace('0b', ''))

Let's start from a base composed by Golden Ration numbers with (0,0), (0,1), (1,0) as coefficients:

1. Add to the base the reciprocals.
2. Add to the base the opposites.
3. Compute the closed set and add its commonest elements to the base to obtain the new base.
4. Do 3. until the base is 128 long.

In the loop, we populate arrays with information about the bases and their closed sets in order to build a pandas dataframe.

In [44]:
base = {Goldice(0,0), Goldice(1,0), Goldice(0,1)}
reciprocals = set()
for elem in base:
    reciprocals.add(elem.reciprocal())
print("Reciprocals:", reciprocals)
base = base.union(reciprocals)
print("Base + Reciprocals:", base)
negatives = set()
for elem in base:
    negatives.add(elem.opposite())
print("Negatives:", negatives)
base = base.union(negatives)
print("Base + Reciprocals + Negatives:", base)

execution_time = 0

base_length_list = []
closed_set_length_list = []
min_range_a = []
max_range_a = []
min_range_a_bits = []
max_range_a_bits = []
min_range_b = []
max_range_b = []
min_range_b_bits = []
max_range_b_bits = []
execution_time_list = []

while len(base) != 128:
    start_time = time.time()
    base = compute_new_base(base)
    end_time = time.time()
    closed_set = compute_closed_set(base)
    a = [elem.a for elem in closed_set if elem != COMPLEX_INFINITY]
    b = [elem.b for elem in closed_set if elem != COMPLEX_INFINITY]
    closed_set_length_list.append(len(closed_set))
    base_length_list.append(len(base))
    min_range_a.append(min(a))
    max_range_a.append(max(a))
    min_range_a_bits.append(bits_needed(min(a)))
    max_range_a_bits.append(bits_needed(max(a)))
    min_range_b.append(min(b))
    max_range_b.append(max(b))
    min_range_b_bits.append(bits_needed(min(b)))
    max_range_b_bits.append(bits_needed(max(b)))
    execution_time = execution_time + end_time - start_time
    execution_time_list.append(execution_time)
    print("-------------------------")
    print(len(base))
    print("Range of coefficient a:", min(a), max(a), len(bin(min(a))), len(bin(max(a))))
    print("Range of coefficient b:", min(b), max(b), len(bin(min(b))), len(bin(max(b))))
    print("Execution time: ", execution_time)


Reciprocals: {(1,0), (complex_infinity,complex_infinity), (-1,1)}
Base + Reciprocals: {(0,0), (1,0), (0,1), (complex_infinity,complex_infinity), (-1,1)}
Negatives: {(0,0), (complex_infinity,complex_infinity), (-1,0), (0,-1), (1,-1)}
Base + Reciprocals + Negatives: {(0,0), (1,0), (0,1), (complex_infinity,complex_infinity), (-1,0), (0,-1), (1,-1), (-1,1)}
-------------------------
12
Range of coefficient a: (-5, 5, 6, 5)
Range of coefficient b: (-3, 3, 5, 4)
Execution time:  0.020536184310913086
-------------------------
16
Range of coefficient a: (-13, 13, 7, 6)
Range of coefficient b: (-8, 8, 7, 6)
Execution time:  0.05191802978515625
-------------------------
20
Range of coefficient a: (-34, 34, 9, 8)
Range of coefficient b: (-21, 21, 8, 7)
Execution time:  0.09474492073059082
-------------------------
24
Range of coefficient a: (-89, 89, 10, 9)
Range of coefficient b: (-55, 55, 9, 8)
Execution time:  0.1573009490966797
-------------------------
28
Range of coefficient a: (-233, 233, 

The pandas dataframe has the following columns:

1. The length of the base.
2. The length of the correspondent closed set.
3. The maximum number of bits needed to represent the goldice (max bits for coefficient a + max bits for coefficient b).
4. The maximum and minimum value for the coefficient a.
5. The maximum and minimum value for the coefficient b.
6. The execution time needed to build the base.

In [52]:
max_bits = [elem1 + elem2 for elem1, elem2 in zip(min_range_a_bits, min_range_b_bits)]
min_max_a = [(elem1, elem2) for elem1, elem2 in zip(min_range_a, max_range_a)]
min_max_b = [(elem1, elem2) for elem1, elem2 in zip(min_range_b, max_range_b)]

d = {'Base Length': base_length_list, 'Closed Set Length': closed_set_length_list, 'Max Bits needed': max_bits, 'Min-Max a': min_max_a, 'Min-Max b': min_max_b, "Execution time (s)": execution_time_list}
df = pd.DataFrame(d)
df = df.set_index('Base Length')
df

,Closed Set Length,Max Bits needed,Min-Max a,Min-Max b,Execution time (s)
Base Length,,,,,
12,78,7,"(-5, 5)","(-3, 3)",0.020536
16,170,10,"(-13, 13)","(-8, 8)",0.051918
20,302,13,"(-34, 34)","(-21, 21)",0.094745
24,474,15,"(-89, 89)","(-55, 55)",0.157301
28,686,18,"(-233, 233)","(-144, 144)",0.293342
32,938,21,"(-610, 610)","(-377, 377)",0.542687
36,1230,23,"(-1597, 1597)","(-987, 987)",0.617284
40,1562,27,"(-4181, 4181)","(-2584, 2584)",0.924273
44,1934,29,"(-10946, 10946)","(-6765, 6765)",1.270638
